In [1]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import talib 
from dateutil.relativedelta import relativedelta
from sklearn.decomposition import PCA
import seaborn as sns
import numpy as np

from Tool_v1 import *


# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler  
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
yf.pdr_override()

In [2]:
start="2020-12-31"
end="2022-12-31"
name="AAPL"

#MACD619RSI21_8020
stock_data=pct_change(name,start,end)
stock_data=stock_data.copy()

#指标运算
stock_data=MACD_short(stock_data)
stock_data=RSI(stock_data)
stock_data=RSI_MACD(stock_data,21,80,20)
stock_data= my_rsi(stock_data, 21)

stock_data = stock_data.dropna()

#y值
#stock_data["y"]=np.where(stock_data['Close'].shift(-1) > stock_data['Close'],1,-1)

num_columns = stock_data.shape[1]+1
print(num_columns)

21


In [3]:
variables=stock_data.loc[:,["re_close", "re_open", "my_rsi", "Position", "RSI6", "up_or_down"]]
variables.head()

KeyError: "['up_or_down'] not in index"

In [ ]:
cor = np.corrcoef(variables.T)
print(cor)
sns.heatmap(cor, annot=True)

In [ ]:
# lr = LogisticRegression()

# 设置正则化类型（L1, L2, Elastic Net）
penalty = 'l2' 

# 设置正则化强度（C）
C = 0.01

#logistic regression
lr = LogisticRegression(penalty =penalty, C=C) 

X = stock_data.loc[:,["re_close", "re_open", "my_rsi", "Position", "RSI21"]].values

st_x= StandardScaler()  
X= st_x.fit_transform(X)  

#Y = stock_data.loc[:,["up_or_down"]].values.ravel()
Y = np.where(stock_data['Open'].shift(-1) > stock_data['Open'],1,-1)
#Y = np.where(stock_data['Close'].shift(-1) > stock_data['Close'],1,-1)

#X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8)

split = int(0.8*len(stock_data))
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]

In [ ]:
lr.fit(X_train, Y_train)
y_pred = lr.predict(X_test)
f1_score(y_pred, Y_test)

In [ ]:
print(lr.score(X_test,Y_test))

In [4]:
start="2020-12-31"
end="2022-12-31"
name="AAPL"

#MACD619RSI21_8020
stock_data=pct_change(name,start,end)
print(stock_data)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-12-31  134.080002  134.740005  131.720001  132.690002  130.387253   
2021-01-04  133.520004  133.610001  126.760002  129.410004  127.164146   
2021-01-05  128.889999  131.740005  128.429993  131.009995  128.736374   
2021-01-06  127.720001  131.050003  126.379997  126.599998  124.402924   
2021-01-07  128.360001  131.630005  127.860001  130.919998  128.647934   
...                ...         ...         ...         ...         ...   
2022-12-23  130.919998  132.419998  129.639999  131.860001  131.127045   
2022-12-27  131.380005  131.410004  128.720001  130.029999  129.307251   
2022-12-28  129.669998  131.029999  125.870003  126.040001  125.339417   
2022-12-29  127.989998  130.479996  127.730003  129.610001  128.889572   
2022-12-30  128.410004  129.949997  127.430000  129.929993  129.207794   

               Volume  Close_price_pe